Bài 3 – Gom cụm (20%)

Trong bài tập này, bạn sẽ làm việc trên tập dữ liệu plants về sự phân bố của một số loài thực vật ở
khu vực Mỹ và Canada (tải dữ liệu từ http://archive.ics.uci.edu/ml/datasets/Plants). Lưu ý: có vùng
phân bố không được liệt kê trong tập tin mô tả dữ liệu, cần đối chiếu với tập tin dữ liệu để xác định
đúng danh sách vùng phân bố.
Bạn cần thực nghiệm giải thuật gom cụm k-Means của MLlib trên dữ liệu được cho. Để hoàn tất
yêu cầu của câu hỏi, các bước cần thực hiện là <br> <br>
1) Đọc các tập tin cần thiết vào PySpark và áp dụng kỹ thuật tiền xử lý dữ liệu phù hợp để
chuyển đổi dữ liệu về dạng vectơ nhị phân. Như vậy ta sẽ có một DataFrame bao gồm:
• Cột đầu tiên thể hiện tên loài thực vật và các cột tiếp theo biểu diễn vùng địa lý.
• Mỗi dòng thể hiện thông tin phân bố địa lý của nột loài thực vật – nếu loài thực vật
có tại một vùng địa lý thì ô tương ứng mang giá trị 1, ngược lại mang giá trị 0.
Lưu dữ liệu sau khi tiền xử lý thành tập tin plants.csv. Đính kèm khi nộp. <br> <br>
2) Chương trình thực hiện gom cụm các loài thực vật theo vùng địa lý bằng giải thuật k-means
và đánh giá kết quả gom cụm bằng ClusteringEvaluator. Thử nghiệm với một số giá trị k.

# Import thư viện

In [ ]:
import pandas as pd
import numpy as np

## Cài đặt pyspark

In [ ]:
!pip install pyspark

In [ ]:
!pip install findspark

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession # required to created a dataframe
spark=SparkSession.builder.appName("Basics").getOrCreate() 


# Đọc dữ liệu

In [ ]:
!ls data

In [ ]:

plants_data = spark.read.load("../../Bai3/data/plants.data", format="csv", header=False, delimiter=",")


In [ ]:
plants_data.show()

# Tiền xử lý

Nối các cột với nhau

In [ ]:
from pyspark.sql.functions import expr, split, explode

df_cau3 = plants_data.withColumn("value", expr(" _c0 || ',' || _c1 || ',' || _c2"))
df_cau3.show()

Tạo cột name và cột location

In [ ]:

df_cau3 = df_cau3.withColumn("row_ar", split("value", ",")) \
  .select( \
          expr("row_ar[0]").alias("name"),
          expr("trim('[]', string(slice(row_ar, 2, size(row_ar) - 1)))").alias("location")
  )

In [ ]:
df_cau3.show()

Tách nội dung ở cột location thành các mảng

In [ ]:
df_cau3 = df_cau3.withColumn('list_sa', split("location", ',')) \
  .select( \
      df_cau3.name,
      expr('list_sa')
  )
df_cau3.show()

Sử dụng explode để khám phá từng giá trị trong mảng

In [ ]:
df_cau3 = df_cau3.select(df_cau3.name, explode(df_cau3.list_sa))

Sử dụng groupby và chuyển về dạng pivot để được các cột với giá trị 1

In [ ]:
from pyspark.sql.functions import lit

df_cau = df_cau3.withColumn('flag', lit(1))
pivotDF = df_cau.groupBy('name').pivot('col').sum()
pivotDF.show()

Điền các giá trị null bằng giá trị 0

In [ ]:
pivotDF = pivotDF.na.fill(value=0)
pivotDF.show()


In [ ]:
print(len(pivotDF.columns))

# Lưu dữ liệu 

In [ ]:
pivotDF.write.csv("../../output/plants_csv")

# Vector Assember các thuộc tính để đưa vào huấn luyện

Tạo vector đặc trưng từ DataFrame
Để tạo vector đặc trưng (feature vector) cho từng dòng dữ liệu, dùng lớp VectorAssembler để ghép giá trị ở các cột trong DataFrame thành một cột mới chứa vector đặc trưng:

In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols = list(pivotDF.columns)[1:], 
                            outputCol="features") 
plants_features = assembler.transform(pivotDF) 
plants_features.show(5)

Lúc này dữ liệu đã sẵn sàng để đưa vào huấn luyện mô hình ML (ở đây là K-means). Gỉa sử chọn số cụm là 30, tiến hành huấn luyện mô hình K-means với PySpark bằng các lệnh sau:

In [ ]:
from pyspark.ml.clustering import KMeans

kmeans = KMeans().setK(30).setSeed(0)
model = kmeans.fit(plants_features)

Hiển thị kết quả

In [ ]:
predictions = model.transform(plants_features)
from pyspark.ml.evaluation import ClusteringEvaluator

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# Thử nhiệm với K = 50, 100, 150, 200

In [ ]:
def test_kmeans(data, k):
  kmeans = KMeans().setK(k).setSeed(0)
  model = kmeans.fit(data)

  predictions = model.transform(data)

  #test
  evaluator = ClusteringEvaluator()
  silhouette = evaluator.evaluate(predictions)
  print("Silhouette with squared euclidean distance = " + str(silhouette))

test_kmeans(plants_features, 5)
test_kmeans(plants_features, 100)
test_kmeans(plants_features, 150)
test_kmeans(plants_features, 200)